[【Python実践】メルカリの売れている商品情報をスクレイピングで集めてみた](https://pepenoheya.blog/mercari-scraping-soldout/)

# 動作テスト

In [9]:
# pip install emoji

Note: you may need to restart the kernel to use updated packages.Collecting emoji
     -------------------------------------- 353.7/353.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350810 sha256=57cc89a5579e4065b009fec5d01b7014a80e664f759f8f92182d1d8ee081b2ab
  Stored in directory: c:\users\amuza\appdata\local\pip\cache\wheels\d5\c5\45\6c3b2e538c10e9667495d6bff243ee0a296c3c246478007df3
Successfully built emoji



In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import pandas as pd

item_ls = []
item_url_ls=[]

#ブラウザの設定
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#ブラウザの起動
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=options)
browser.implicitly_wait(3)

#キーワード設定
KEYWORD = 'ラウドマウス'

C:\Users\amuza\AppData\Local\Temp\ipykernel_9992\1839469086.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=options)


In [ ]:
#売り切れ表示
url = 'https://jp.mercari.com/search?keyword=' + KEYWORD + '&status=sold_out%7Ctrading'
browser.get(url)
browser.implicitly_wait(5)

In [ ]:
# 「次へ」をクリックする
next_button = browser.find_element(By.XPATH, '//button[contains(text(), "次へ")]')
next_button.send_keys(Keys.ENTER)

In [ ]:
#商品の詳細ページを開く
item_url = browser.find_elements(By.CSS_SELECTOR, '#item-grid > ul > li a')[0].get_attribute('href')
print(item_url)
browser.get(item_url)

In [ ]:
# 商品ページのURLを取得する
items = browser.find_element(By.XPATH, '//li[@data-testid="item-cell"]//a')
print(items.text)
print(items.get_attribute('href'))

In [ ]:
# 詳細ページのURLリストを作る
detail_url_list = []

def get_detail_url():
    items = browser.find_elements(By.XPATH, '//li[@data-testid="item-cell"]')

    for elem in items:
        detail_url = elem.find_element(By.XPATH, './/a').get_attribute('href')
        # print(detail_url)
        detail_url_list.append(detail_url)

    print(detail_url_list)
    
get_detail_url()

In [ ]:
# 詳細ページの中の情報を取得する
data_list = []

def get_data():
    for detail_url in detail_url_list:
        # print(detail_url)
        browser.get(detail_url)
        browser.implicitly_wait(3)
        item_name = browser.find_element(By.XPATH, '//h1').text.replace(',', '')
        item_price = int(browser.find_element(By.XPATH, '(//div[@data-testid="price"]/span)[2]').text.replace(',', ''))
        item_tax_postage = browser.find_element(By.XPATH, '//p[@class="merText caption__ddd9b9a9 primary__ddd9b9a9 sc-d7d6f25a-2 fuRYAY"]').text
        # print(item_name)
        # print(item_price, '円', item_tax_postage)
        
        data = {
            '商品名': item_name,
            '価格': item_price,
            '税区分,送料': item_tax_postage
            }
        data_list.append(data)
        print(data_list)

get_data()

In [ ]:
item_name = browser.find_element(By.CSS_SELECTOR,'h1').text
# shadow_root = browser.find_element(By.CSS_SELECTOR,'#item-info > section:nth-child(2) > mer-show-more').shadow_root
# item_ex = shadow_root.find_element(By.CSS_SELECTOR,'div.content.clamp').text
# src_shadow = browser.find_element(By.CSS_SELECTOR,'#main > article > div.layout__LayoutCol-sc-1lyi7xi-2.jNCCiQ > section > div > div > div > div > div.layout__LayoutCol-sc-1lyi7xi-2.cHKFjh > div > div > div > div > div.slick-slide.slick-active.slick-current > div > div > mer-item-thumbnail').shadow_root
# src = src_shadow.find_element(By.CSS_SELECTOR,'div > figure > div.image-container > picture > img').get_attribute('src')
# shadow_root1 = browser.find_element_by_css_selector('#item-grid > ul > li:nth-child(1) > a > mer-item-thumbnail').shadow_root
# price_shadow = shadow_root1.find_element(By.CSS_SELECTOR,'div > figure > div.price-container > mer-price').shadow_root
# item_price = price_shadow.find_element(By.CSS_SELECTOR,'div[data-testid="price"] > span')[0].text
item_price = browser.find_elements(By.CSS_SELECTOR,'div[data-testid="price"] > span')[1].text.replace(',', '')
item_status = browser.find_element(By.CSS_SELECTOR,'div[data-testid="checkout-button"] > button').text # 販売状況
item_post_time  = browser.find_element(By.CSS_SELECTOR,'p.merText.body__ddd9b9a9.secondary__ddd9b9a9').text

print('item_name:', item_name)
# print('shadow_root:', shadow_root)
# print('item_ex:',item_ex) 
# print('src_shadow:',src_shadow) 
# print('src:',src) 
# print('shadow_root1:',shadow_root1) 
# print('price_shadow:',price_shadow) 
print('item_price:',item_price) 
print('item_status:',item_status) 
print('item_post_time:',item_post_time) 

In [2]:
# データ保存
data_list = []

for i in range(2):
    data = {
        '商品名': f'item_name{i}',
        '価格': f'item_price{i}',
        '税区分,送料': f'item_tax_postage{i}'
        }
    data_list.append(data)
    
output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'メルカリデータ_' + output_time + '.csv'
pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

In [10]:
# 絵文字の削除
# Pythonで絵文字を判定・削除する方法
# https://wagtailbase.com/article/python-emoji/
import emoji

string = "✨ラウドマウスゴルフ✨カラフルハーフパンツ　レディースUS/0"
print(string)

# 絵文字を削除する
no_emoji_string = emoji.replace_emoji(string)
print(no_emoji_string)

✨ラウドマウスゴルフ✨カラフルハーフパンツ　レディースUS/0
ラウドマウスゴルフカラフルハーフパンツ　レディースUS/0


# 完成版（自作）

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import pandas as pd

detail_url_list = [] # 詳細ページURL格納用リスト
data_list = [] # データ格納用リスト

#ブラウザの設定
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#ブラウザの起動
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=options)
browser.implicitly_wait(3)

# 検索キーワード設定
KEYWORD = 'ラウドマウス'

# スクレイピング用URL
url = 'https://jp.mercari.com/search?keyword=' + KEYWORD # デフォルト検索結果
# url = 'https://jp.mercari.com/search?keyword=' + KEYWORD + '&status=sold_out%7Ctrading' # 売り切れ情報
# url = 'https://jp.mercari.com/search?keyword=' + KEYWORD + '&status=on_sale' # 販売中情報
browser.get(url)
browser.implicitly_wait(3)

# 詳細ページのURLリスト作成
def get_detail_url():
    items = browser.find_elements(By.XPATH, '//li[@data-testid="item-cell"]')

    for elem in items:
        detail_url = elem.find_element(By.XPATH, './/a').get_attribute('href')
        # print(detail_url)
        detail_url_list.append(detail_url)

# 詳細ページの中の情報を取得する
def get_data():
    i = 1
    for detail_url in detail_url_list:
        # print(detail_url)
        browser.get(detail_url)
        browser.implicitly_wait(3)

        try:
            item_name = emoji.replace_emoji(browser.find_element(By.XPATH, '//h1').text.replace(',', '')) # 商品名（絵文字削除）
            item_price = int(browser.find_element(By.XPATH, '(//div[@data-testid="price"]/span)[2]').text.replace(',', '')) # 価格
            item_tax_postage = browser.find_element(By.XPATH, '//p[@class="merText caption__ddd9b9a9 primary__ddd9b9a9 sc-d7d6f25a-2 fuRYAY"]').text # 税,送料
            item_sales_status = browser.find_element(By.XPATH,'//div[@data-testid="checkout-button"]/button').text.replace('購入手続きへ', '販売中').replace('売り切れました', '売り切れ') # 販売状況
        except:
            error_msg = '取得失敗'
            item_name = error_msg
            item_price = error_msg
            item_tax_postage = error_msg
            item_sales_status = error_msg
        finally:
            print(i, ':', item_name)
            # print(item_price, '円', item_tax_postage)

        data = {
            '商品名': item_name,
            '価格': item_price,
            '税区分,送料': item_tax_postage,
            '販売状況': item_sales_status,
            'URL': detail_url
            }
        data_list.append(data)
        # print(data_list)
        i = i + 1

# データ保存
def output_data():
    output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    csv_file_name = 'メルカリデータ_' + KEYWORD + '_' + output_time + '.csv'
    pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
    
def main():
    get_detail_url()
    get_data()
    output_data()

if __name__ == '__main__':
    main()

C:\Users\amuza\AppData\Local\Temp\ipykernel_9992\2938842106.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=options)


1 : ラウドマウス 半袖ポロシャツ シャガデリック柄 ホワイト L 新品
2 : ラウドマウス 半袖ポロシャツ 星 ボーダー柄 ネイビー L 【新品】
3 : ラウドマウスゴルフカラフルハーフパンツ　レディースUS/0
4 : 新品タグ付き　ラウドマウス スカートS
5 : Ｍサイズ　ラウドマウスレディースUVカットモックネック半袖シャツ　サンプル品
6 : パーリーゲイツモック　ラウドマウスハーフパンツ　２点セット
7 : ラウドマウス　半袖ポロシャツ　ブルーフラミンゴ　Lサイズ
8 : ラウドマウス　スカート
9 : ラウドマウス　半袖　ポロシャツ M
10 : ラウドマウス ゴルフウェア レディース 半袖 ポロシャツ ブルー M
11 : ラウドマウス レディース ポロシャツ オレンジイエロー M 春夏 ウエア 半袖
12 : 取得失敗
13 : LOUDMOUTH ラウドマウス Sサイズ 半袖ポロシャツ レディース
14 : LOUDMOUTH ラウドマウス ワンポイント総柄半袖ポロシャツ ネイビー
15 : Loudmouth ラウドマウス　スカート　レディースM アメリカ　国旗　ミニ
16 : 【新品｝Loudmouth ラウドマウス　長袖　ポロシャツ
17 : ラウドマウス　ゴルフスカート
18 : LOUDMOUTH ラウドマウス　スカートS
19 : LOUDMOUTH ラウドマウス　ポロシャツ　レディース　M
20 : 【新品】LOUDMOUTH　ラウドマウス　ゴルフレインウェア
21 : ラウドマウス　レディース　ゴルフ　キャップ　スカート　2点セット　M ★美品★
22 : 【ラウドマウス】 ゴルフウェア ノースリーブ オレンジ
23 : LOUDMOUTH ラウドマウス　ロゴデザイン入り　ゴルフ　ポロシャツ　M
24 : LOUDMOUTH.ラウドマウス　シャークデザイン入り　ゴルフ　ポロシャツ　M
25 : LOUDMOUTH ラウドマウス　ロゴ入り　薄目総柄デザイン　ゴルフパンツ　Ｍ
26 : LOUDMOUTH ラウドマウス　総柄デザイン　ロゴ入り　ゴルフパンツ　M
27 : ラウドマウス 総柄 パンツ
28 : 新品 ラウドマウス インナー付 スカート スイートハート柄
29 : 【LOUDMOUTH】ラウドマウス ゴルフ サンバ

# 完成版（サイトコピペ）

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import time
import pandas as pd

item_ls = []
item_url_ls=[]

#ブラウザの設定
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#ブラウザの起動
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=options)
browser.implicitly_wait(3)

#キーワード設定
KEYWORD = '白Tシャツ メンズ'

def get_url():
    #売り切れ表示
    url = 'https://jp.mercari.com/search?keyword=' + KEYWORD + '&status=sold_out%7Ctrading'
    browser.get(url)
    browser.implicitly_wait(5)

    #商品の詳細ページのURLを取得する
    item_box = browser.find_elements_by_css_selector('#item-grid > ul > li')
    for item_elem in item_box:
        item_url_ls.append(item_elem.find_element_by_css_selector('a').get_attribute('href'))

def get_data():
    #商品情報の詳細を取得する
    for item_url in item_url_ls:
        browser.get(item_url)
        time.sleep(3)
        #商品名 
        item_name = browser.find_element(By.CSS_SELECTOR,'#item-info > section:nth-child(1) > div.mer-spacing-b-12 > mer-heading').text
        shadow_root = browser.find_element(By.CSS_SELECTOR,'#item-info > section:nth-child(2) > mer-show-more').shadow_root
        item_ex = shadow_root.find_element(By.CSS_SELECTOR,'div.content.clamp').text
        src_shadow = browser.find_element(By.CSS_SELECTOR,'#main > article > div.layout__LayoutCol-sc-1lyi7xi-2.jNCCiQ > section > div > div > div > div > div.layout__LayoutCol-sc-1lyi7xi-2.cHKFjh > div > div > div > div > div.slick-slide.slick-active.slick-current > div > div > mer-item-thumbnail').shadow_root
        src = src_shadow.find_element(By.CSS_SELECTOR,'div > figure > div.image-container > picture > img').get_attribute('src')
        shadow_root1 = browser.find_element_by_css_selector('#item-grid > ul > li:nth-child(1) > a > mer-item-thumbnail').shadow_root
        price_shadow = shadow_root1.find_element(By.CSS_SELECTOR,'div > figure > div.price-container > mer-price').shadow_root
        item_price = price_shadow.find_element(By.CSS_SELECTOR,'span.number').text
        
        data = {
            '商品名':item_name,
            '商品説明':item_ex,
            '価格':item_price,
            'URL':item_url,
            '画像URL':src
        }

        item_ls.append(data)

def main():
    get_url()
    get_data()
    pd.DataFrame(item_ls).to_csv('メルカリデータ.csv')

if __name__ == '__main__':
    main()